In [1]:
%pylab inline
from astropy.table import Table
from rnn import RNNClassifier
import tensorflow as tf

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Loading dataset
data_table = Table.read('normalized_no_clip.fits')

In [3]:
data_table

FILENAME,SURVEY,SNID,SNTYPE,FILTERS,RA,DECL,FAKE,MWEBV,REDSHIFT_SPE,NOBS,NVAR,"DATA [50,9]",obs_length
bytes16,bytes7,bytes12,bytes5,bytes6,bytes21,bytes20,bytes35,bytes23,bytes25,bytes5,bytes3,float64,float64
DES_SN000018.DAT,DES,18,-9,griz,34.500000 deg,-5.500000 deg,3 (=> BLIND-TEST simulation),0.0227 MW E(B-V),-9.0000 +- 9.0000,118,5,0.0 .. 0.0,0.8341289942655092
DES_SN000050.DAT,DES,50,-9,griz,36.750000 deg,-4.500000 deg,3 (=> BLIND-TEST simulation),0.0283 MW E(B-V),-9.0000 +- 9.0000,16,5,0.0 .. 0.0,0.17683683328855004
DES_SN000051.DAT,DES,51,1,griz,52.500000 deg,-27.500000 deg,3 (=> BLIND-TEST simulation),0.0097 MW E(B-V),0.5177 +- 0.0050,121,5,0.0 .. 0.0,0.7254312291675129
DES_SN000063.DAT,DES,63,-9,griz,34.500000 deg,-5.500000 deg,3 (=> BLIND-TEST simulation),0.0227 MW E(B-V),-9.0000 +- 9.0000,115,5,0.0 .. 0.0,0.731480158026834
DES_SN000064.DAT,DES,64,-9,griz,48.000000 deg,0.000000 deg,3 (=> BLIND-TEST simulation),0.0713 MW E(B-V),-9.0000 +- 9.0000,92,5,0.0 .. 0.0,0.6740610724556567
DES_SN000096.DAT,DES,96,-9,griz,34.500000 deg,-5.500000 deg,3 (=> BLIND-TEST simulation),0.0227 MW E(B-V),-9.0000 +- 9.0000,104,5,0.0 .. 0.0,0.6853870914257367
DES_SN000122.DAT,DES,122,-9,griz,34.500000 deg,-5.500000 deg,3 (=> BLIND-TEST simulation),0.0227 MW E(B-V),-9.0000 +- 9.0000,11,5,0.0 .. 0.0,0.10264311891463129
DES_SN000124.DAT,DES,124,-9,griz,48.000000 deg,0.000000 deg,3 (=> BLIND-TEST simulation),0.0713 MW E(B-V),-9.0000 +- 9.0000,95,5,0.0 .. 0.0,0.6515748369133555
DES_SN000156.DAT,DES,156,-9,griz,36.750000 deg,-4.500000 deg,3 (=> BLIND-TEST simulation),0.0283 MW E(B-V),-9.0000 +- 9.0000,87,5,0.0 .. 0.0,0.5490403471558932


In [49]:
# Splitting training and testing data
randomize_inds = range(len(data_table))
randomize_inds = permutation(randomize_inds)
randomized_inds_train = randomize_inds[0:15000]
randomized_inds_test  = randomize_inds[15000:]

In [50]:
data_table['label'] = 1.0*(data_table['SNTYPE'].astype('int') == 1)

In [51]:
# Define input function for training 
def input_fn_train():
    def mapping_function(x):
        def extract_batch(inds):
            inds = randomized_inds_train[inds]
            return data_table['label'][inds].astype('float32'), clip(data_table['obs_length'][inds],0,49).astype('int32'),data_table['DATA'][inds].astype('float32') 
        a,b,c =tf.py_func( extract_batch, [x], [tf.float32, tf.int32, tf.float32])
        a.set_shape([None])
        b.set_shape([None])
        c.set_shape([None,50,9])
        return a,b,c

    dataset = tf.data.Dataset.range(len(randomized_inds_train))
    dataset = dataset.repeat().shuffle(10000).batch(512)
    dataset = dataset.map( mapping_function)
    iterator = dataset.make_one_shot_iterator()
    label, length, ts = iterator.get_next()
    return  {'length':length, 'ts':ts}, label

def input_fn_test():
    def mapping_function(x):
        def extract_batch(inds):
            inds = randomized_inds_test[inds]
            return data_table['label'][inds].astype('float32'), clip(data_table['obs_length'][inds],0,49).astype('int32'),data_table['DATA'][inds].astype('float32') 
        a,b,c =tf.py_func( extract_batch, [x], [tf.float32, tf.int32, tf.float32])
        a.set_shape([None])
        b.set_shape([None])
        c.set_shape([None,50,9])
        return a,b,c

    dataset = tf.data.Dataset.range(len(randomized_inds_test))
    dataset = dataset.batch(512)
    dataset = dataset.map( mapping_function)
    iterator = dataset.make_one_shot_iterator()
    label, length, ts = iterator.get_next()
    return  {'length':length, 'ts':ts}, label

In [52]:
model = RNNClassifier(hidden_units=[128, 128], dropout=0.5, model_dir='models/test6')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models/test6', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x126f79cc0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [53]:
model.train(input_fn=input_fn_train, steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/test6/model.ckpt-1500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1501 into models/test6/model.ckpt.
INFO:tensorflow:loss = 0.463623, step = 1501
INFO:tensorflow:global_step/sec: 21.9106
INFO:tensorflow:loss = 0.44964468, step = 1601 (4.565 sec)
INFO:tensorflow:global_step/sec: 23.637
INFO:tensorflow:loss = 0.43219572, step = 1701 (4.228 sec)
INFO:tensorflow:global_step/sec: 25.1685
INFO:tensorflow:loss = 0.42423537, step = 1801 (3.976 sec)
INFO:tensorflow:global_step/sec: 26.1077
INFO:tensorflow:loss = 0.4017268, step = 1901 (3.828 sec)
INFO:tensorflow:global_step/sec: 26.1588
INFO:tensorflow:loss = 0.38285547, step = 2001 (3.823 sec)
INFO:tensorflow:global_step/sec: 26.8568
INFO:tensorflow:loss = 0.3705176, 

INFO:tensorflow:global_step/sec: 24.1489
INFO:tensorflow:loss = 0.20724133, step = 9401 (4.143 sec)
INFO:tensorflow:global_step/sec: 25.1308
INFO:tensorflow:loss = 0.2012732, step = 9501 (3.978 sec)
INFO:tensorflow:global_step/sec: 26.8923
INFO:tensorflow:loss = 0.21323816, step = 9601 (3.719 sec)
INFO:tensorflow:global_step/sec: 25.2395
INFO:tensorflow:loss = 0.17729287, step = 9701 (3.962 sec)
INFO:tensorflow:global_step/sec: 22.4064
INFO:tensorflow:loss = 0.21322787, step = 9801 (4.462 sec)
INFO:tensorflow:global_step/sec: 21.8812
INFO:tensorflow:loss = 0.23121998, step = 9901 (4.572 sec)
INFO:tensorflow:global_step/sec: 21.5437
INFO:tensorflow:loss = 0.135393, step = 10001 (4.640 sec)
INFO:tensorflow:global_step/sec: 23.9691
INFO:tensorflow:loss = 0.16504693, step = 10101 (4.172 sec)
INFO:tensorflow:global_step/sec: 22.1871
INFO:tensorflow:loss = 0.1293016, step = 10201 (4.507 sec)
INFO:tensorflow:global_step/sec: 22.3157
INFO:tensorflow:loss = 0.2192631, step = 10301 (4.483 sec)
I

In [54]:
# Apply model to training set
test_prob = [p['prob'] for p in model.predict(input_fn_test)]
table_test = data_table[randomized_inds_test]
table_test['p'] = test_prob

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/test6/model.ckpt-11500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [55]:
table_test

FILENAME,SURVEY,SNID,SNTYPE,FILTERS,RA,DECL,FAKE,MWEBV,REDSHIFT_SPE,NOBS,NVAR,"DATA [50,9]",obs_length,label,p
bytes16,bytes7,bytes12,bytes5,bytes6,bytes21,bytes20,bytes35,bytes23,bytes25,bytes5,bytes3,float64,float64,float64,float32
DES_SN115114.DAT,DES,115114,-9,griz,34.500000 deg,-5.500000 deg,3 (=> BLIND-TEST simulation),0.0227 MW E(B-V),-9.0000 +- 9.0000,107,5,0.0 .. 0.0,0.7428747848856917,0.0,0.044443976
DES_SN136659.DAT,DES,136659,-9,griz,52.500000 deg,-27.500000 deg,3 (=> BLIND-TEST simulation),0.0097 MW E(B-V),-9.0000 +- 9.0000,141,5,0.0 .. 0.0,0.8287261230254077,0.0,0.044443976
DES_SN110194.DAT,DES,110194,-9,griz,52.500000 deg,-27.500000 deg,3 (=> BLIND-TEST simulation),0.0097 MW E(B-V),-9.0000 +- 9.0000,101,5,0.0 .. 0.0,0.6635469135026134,0.0,0.044443976
DES_SN126520.DAT,DES,126520,-9,griz,52.500000 deg,-27.500000 deg,3 (=> BLIND-TEST simulation),0.0097 MW E(B-V),-9.0000 +- 9.0000,141,5,0.0 .. 0.0,0.8287261230254077,0.0,0.044443976
DES_SN130173.DAT,DES,130173,-9,griz,52.500000 deg,-27.500000 deg,3 (=> BLIND-TEST simulation),0.0097 MW E(B-V),-9.0000 +- 9.0000,32,5,0.0 .. 0.0,0.2290074153693132,0.0,0.044443976
DES_SN196912.DAT,DES,196912,-9,griz,36.750000 deg,-4.500000 deg,3 (=> BLIND-TEST simulation),0.0283 MW E(B-V),-9.0000 +- 9.0000,88,5,0.0 .. 0.0,0.5946874624800601,0.0,0.044443976
DES_SN116278.DAT,DES,116278,-9,griz,52.500000 deg,-27.500000 deg,3 (=> BLIND-TEST simulation),0.0097 MW E(B-V),-9.0000 +- 9.0000,121,5,0.0 .. 0.0,0.6464406799041961,0.0,0.044443976
DES_SN113327.DAT,DES,113327,-9,griz,48.000000 deg,0.000000 deg,3 (=> BLIND-TEST simulation),0.0713 MW E(B-V),-9.0000 +- 9.0000,36,5,0.0 .. 0.0,0.35918516697445385,0.0,0.044443976
DES_SN125767.DAT,DES,125767,-9,griz,36.750000 deg,-4.500000 deg,3 (=> BLIND-TEST simulation),0.0283 MW E(B-V),-9.0000 +- 9.0000,33,5,0.0 .. 0.0,0.38187722618304665,0.0,0.044443976
